# **Studi Kasus Pembelajaran Mendalam untuk Masalah Regresi: Prediksi Harga Perumahan Boston**

Dalam tugas ini kita akan menemukan bagaimana mengembangkan dan mengevaluasi model jaringan saraf menggunakan Keras untuk masalah regresi. Tugas ini melakukan eksplorasi untuk menjawab pertanyaan-pertanyaan berikut:

*   Berapa banyaknya hidden layer yang optimal?
*   Berapa banyaknya hidden unit yang optimal di setiap hidden layer?
*   Apa activation function di setiap layer sehingga hasilnya optimal?
*   Dari semua pilihan optimizer, apa optimizer yang hasilnya optimal?
*   Dari semua pilihan loss function, apa yang hasilnya optimal?

Dataset yang digunakan pada model prediktif ini sumbernya StatLib library tentang nilai perumahan di pinggiran kota Boston.



Berikut adalah fitur yang dipakai untuk prediksi harga rumah:

1. CRIM : tingkat kejahatan per kapita menurut kota
2. ZN : proporsi tanah perumahan yang dikategorikan untuk kavling lebih dari 25.000 sq.ft.(sekitar 2.300 meter persegi)
3. INDUS : proporsi acre bisnis non-ritel per kota
4. CHAS : Variabel dummy Sungai Charles (= 1 jika saluran membatasi sungai; 0 sebaliknya)
5. NOX : konsentrasi oksida nitrat (bagian per 10 juta)
6. RM : rata-rata jumlah kamar per hunian
7. USIA : proporsi unit yang ditempati oleh pemilik yang dibangun sebelum tahun 1940
8. DIS : jarak tertimbang (weighted distance) ke lima pusat pekerjaan Boston
9. RAD : indeks aksesibilitas ke jalan raya radial
10. PAJAK : tarif pajak properti nilai penuh per $10.000
11. PTRATIO : rasio murid-guru menurut kota
12. B : 1000(Bk - 0.63)^2 dimana Bk adalah proporsi kulit hitam menurut kota
13. LSTAT : % status penduduk yang lebih rendah

In [2]:
# Load libraries
import numpy as np
import pandas as pd

# keras library
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# scikit functions
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load dataset
dataframe = pd.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
dataframe.head()     # preview the first 5 rows

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Di bawah ini, kami mendefinisikan fungsi untuk membuat model dasar yang akan dievaluasi. Model dasar ini adalah model sederhana yang memiliki satu lapisan tersembunyi yang terhubung penuh, dengan jumlah neuron yang sama dengan atribut input (13). Jaringan menggunakan fungsi aktivasi penyearah untuk lapisan tersembunyi. Tidak ada fungsi aktivasi yang digunakan untuk lapisan keluaran karena ini merupakan masalah regresi, dan kami tertarik untuk memprediksi nilai numerik secara langsung tanpa transformasi.

Algoritma optimasi ADAM yang efisien digunakan dan *loss function mean square error* (mse) dioptimalkan. Ini akan menjadi metrik yang sama yang akan kita gunakan untuk mengevaluasi kinerja model. Ini adalah metrik yang diinginkan karena dengan mengambil akar kuadrat dari nilai kesalahan, itu memberi kita hasil yang dapat kita pahami secara langsung, dalam konteks masalah harga rumah (dalam satuan ribuan dolar).

In [3]:
# define base mode
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)

kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


Baseline: -35.67 (21.37) MSE


# Menstandardkan dataset untuk menaikkan performa
Pada dataset harga rumah Boston semua atribut input bervariasi dalam skalanya, karena mereka mengukur kuantitas yang berbeda.

In [4]:
#evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """


Standardized: -28.60 (29.00) MSE


Setelah dataset distandardkan hasilnya memberikan kinerja yang lebih baik dibandingkan model dasar tanpa data standar, yaitu mengurangi kesalahan sekitar 10 ribu dolar kuadrat.
Nilai MSE sebelum data distandarkan: -38.32
Nilai MSE setelah data distandarkan: -27.17

# Eksplorasi Topologi Jaringan yang Lebih Dalam


**Berapa banyaknya hidden layer yang optimal?**

Salah satu cara untuk meningkatkan kinerja jaringan saraf adalah dengan menambahkan lebih banyak lapisan. Ini memungkinkan model untuk mengekstrak dan menggabungkan kembali fitur tingkat tinggi yang disematkan dalam data. Di bagian ini, kita akan mengevaluasi efek penambahan satu lapisan tersembunyi lagi ke model. Ini semudah mendefinisikan fungsi baru yang akan membuat model yang lebih dalam ini, disalin dari model dasar kami di atas. Kita kemudian dapat menyisipkan baris baru setelah lapisan tersembunyi pertama. Dalam hal ini dengan sekitar setengah jumlah neuron. Topologi jaringan kami sekarang terlihat seperti:

13 masukan -> [13 -> 6] -> 1 keluaran

Kita dapat mengevaluasi topologi jaringan ini dengan cara yang sama seperti di atas, sementara juga menggunakan standarisasi dataset yang ditunjukkan di atas untuk meningkatkan kinerja.

In [5]:
# define the model
def larger_model_dua_hidden():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(6, activation='relu', kernel_initializer='normal'))
    model.add(Dense(6, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn = larger_model_dua_hidden, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Larger: -63.58 (123.63) MSE


Percobaan diperbanyak untuk mengamati pengaruh jumlah hidden layer terhadap kinerjanya. Berikut hasil penambahan jumlah hidden layer:

*   1 hideen layer -> MSE = -23.63 dengan nilai varian 26.74
*   2 hidden layer -> MSE = -21.69 dengan nilai varian 25.29
*   3 hidden layer -> MSE = -21.95 dengan nilai varian 23.79
*   4 hidden layer -> MSE = -36.87 dengan nilai varian 56.45

Dari hasil percobaan menunjukan bahwa 2 hidden layer menghasilkan nilai yang optimal. Penambahan jumlah hidden layer menunjukkan bahwa jaringan yang lebih dalam, yang juga lebih mahal secara komputasi tidak selalu lebih baik dalam hal kinerja dan akurasi.

**Berapa banyaknya hidden unit yang optimal di setiap hidden layer?**

Untuk mencari nilai yang optimal untuk jumlah unit dalam hidden layer maka perlu dilakukan percobaan variasi jumlah unit dalam hidden layer. Berikut variasi yang dilakukan:Jumlah unit 13, 16, 20

In [6]:
# define wider model
def wider_model_b():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(12, activation='relu', kernel_initializer='normal'))
    model.add(Dense(8, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model_b, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Wider: -22.05 (26.41) MSE


Ketiga model tersebut menghasilkan nilai sebagai berikut:
model_wider_a dengan jumlah unit 13,10,6 -> MSE: -21.65
model_wider_b dengan jumlah unit 16,12,8 -> MSE: -20.04
model_wider_c dengan jumlah unit 20,12,8 -> MSE: -21.31

Hasil tersebut menunjukkan bahwa variasi jumlah unit pada hidden layer tidak meningkatkan performa secara signifikan. Model_wider_b menghasilkan performa yang sedikit membaik.

**Apa activation function di setiap layer sehingga hasilnya optimal?**

Fungsi aktivasi yang sesuai untuk permasalahan ini dapat dicari dengan mencoba ReLu , Sigmoid dan Tanh

In [7]:
# define wider model
def wider_model_b():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(12, activation='relu', kernel_initializer='normal'))
    model.add(Dense(8, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model_b, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Wider: -20.16 (23.54) MSE


Hasilnya menunjukkan bahwa fungsi aktivasi ReLu memberikan hasil yang terbaik.
Activation function ReLu -> MSE = -20.04
Activation function Sigmoid -> MSE = -61.58
Activation function Tanh -> MSE = -79.28

**Dari semua pilihan optimizer, apa optimizer yang hasilnya optimal?**





In [8]:
# define wider model
def wider_model_b():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(12, activation='relu', kernel_initializer='normal'))
    model.add(Dense(8, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model_b, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Wider: -20.67 (24.66) MSE


Hasil dari beberapa pilihan optimizer yang digunakan adalah sebagai berikut:
*   SGD: MSE -83.83 (varian 60.57)
*   ADAGRAD: MSE -349.86 (varian 194.00)
*   RMSPROP: MSE -21.26 (varian 25.73)
*   ADAM: MSE -20.62 (varian 25.24)

Dari hasil tersebut nampak optimizer yang memberikan hasil terbaik adalah ADAM dan RMSPROP.


**Dari semua pilihan loss function, apa yang hasilnya optimal?**


Pilihan loss function dapat berupa:
*   Mean Squared Error (MSE)
*   Mean Absolute Error (MAE)
*   Huber Loss







In [9]:
# define wider model
def wider_model_b():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, activation='relu', kernel_initializer='normal'))
    model.add(Dense(12, activation='relu', kernel_initializer='normal'))
    model.add(Dense(8, activation='relu', kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='ADAM')
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model_b, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Wider: -20.04 (22.55) MSE


Hasil dari beberapa pilihan loss sebagai berikut:
*   Mean Squared Error (MSE) : -20.62 (varian 25.24)
*   Mean Absolute Error (MAE) : -3.01 (varian 1.30)
*   Huber loss : -2.51 (varian 1.50)

